In [1]:
import pandas as pd
import datetime

In [2]:
Bestand_SteA_long = pd.read_csv('Bestand_SteA_long.csv', index_col=0)
AST_long = pd.read_csv('AST_long.csv', index_col=0)
SvB_AO_long = pd.read_csv('SvB_AO_long.csv', index_col=0)
SvB_WO_long = pd.read_csv('SvB_WO_long.csv', index_col=0)

## merge

In [3]:
df_final_wide = pd.read_csv('../../final_dfs/for_modeling/df_final_date_wide_2007.csv', index_col=0)
df_final_long = pd.read_csv('../../final_dfs/for_modeling/df_final_date_long_2007.csv', index_col=0)

In [4]:
#set(df_final_long['variable'])

In [5]:
# fix date to first day of the month to merge
def fix_day(date):
    return date.replace(day=1)

### wide

In [6]:
df_final_wide.shape

(67368, 40)

In [7]:
def wide_merge(df_wide, df_final_wide, varname):
    df_wide['datetime'] = pd.to_datetime(df_wide['time_stamp'], format='%Y-%m-%d')
    df_wide['date'] = df_wide['datetime'].apply(fix_day)
    df_wide = df_wide[['ags5', 'date', 'value']]
    df_wide.columns = ['ags5', 'date', f'{varname}']
    df_final_wide['date'] = pd.to_datetime(df_final_wide['date'])
    df_final_wide = df_final_wide.drop(columns=[f'{varname}'])
    df_final_wide = pd.merge(df_final_wide, df_wide, on=['ags5', 'date'])
    return df_final_wide

In [8]:
df_final_wide = wide_merge(Bestand_SteA_long, df_final_wide, 'registerd_jobs')
df_final_wide = wide_merge(AST_long, df_final_wide, 'unemployed')
df_final_wide = wide_merge(SvB_AO_long, df_final_wide, 'employees_social_security_at_work')
df_final_wide = wide_merge(SvB_WO_long, df_final_wide, 'employees_social_security_at_residence')

In [9]:
df_final_wide.shape

(65363, 40)

### long

In [10]:
df_final_long.shape

(599980, 5)

In [11]:
def long_merge(df_long, df_final_long, varname):
    df_final_long['date'] = pd.to_datetime(df_final_long['date'])
    df_final_long = df_final_long[df_final_long['variable']!=varname]
    df_long['ags2'] = df_long['ags5'].astype(str).str[:-3]
    df_long.rename(columns={f'{varname}': "value"}, inplace=True)
    df_long['variable'] = f'{varname}'
    df_long = df_long[['ags2', 'ags5', 'variable', 'date', 'value']]
    df_final_long = pd.concat([df_final_long, df_long], ignore_index=True)
    return df_final_long

In [12]:
df_final_long = long_merge(Bestand_SteA_long, df_final_long, 'registerd_jobs')
df_final_long = long_merge(AST_long, df_final_long, 'unemployed')
df_final_long = long_merge(SvB_AO_long, df_final_long, 'employees_social_security_at_work')
df_final_long = long_merge(SvB_WO_long, df_final_long, 'employees_social_security_at_residence')

In [13]:
df_final_long.shape

(855016, 5)

In [14]:
# df_final_wide.to_csv('../../final_dfs/for_modeling/df_final_date_wide_2007.csv')
# df_final_long.to_csv('../../final_dfs/for_modeling/df_final_date_long_2007.csv')